In [2]:
import azureml.core

from azureml.core import Experiment, Workspace, Dataset, Datastore
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

StatementMeta(, , , SessionError, )

InvalidHttpRequestToLivy: Your Spark job requested 12 vcores. However, the pool only has 4 vcores available out of quota of 16 vcores. Try ending the running job(s) in the pool, reducing the numbers of vcores requested, increasing the pool maximum size or using another pool. HTTP status code: 400. Trace ID: d4ebf429-402b-43aa-9b38-2b60a26103c4.

In [ ]:
subscription_id = "your_subscription_id"
resource_group = "your_resource_group"
workspace_name = "your_mlworkspace"
experiment_name = "experimentname"

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
experiment = Experiment(ws, experiment_name)

In [ ]:
df = spark.sql("SELECT * FROM default.activitysummary")

datastore = Datastore.get_default(ws)
dataset = TabularDatasetFactory.register_spark_dataframe(df, datastore, name = experiment_name + "-dataset")

StatementMeta(, , , Cancelled, )

In [ ]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(
    time_column_name = "dateComponents",
    forecast_horizon = "auto",
    time_series_id_column_names = ["appleExerciseTime","appleStandHours"],
)

automl_config = AutoMLConfig(spark_context = sc,
                             task = "forecasting",
                             training_data = dataset,
                             label_column_name = "activeEnergyBurned",
                             primary_metric = "normalized_root_mean_squared_error",
                             experiment_timeout_hours = 3,
                             max_concurrent_iterations = 2,
                             n_cross_validations = 5,
                             forecasting_parameters = forecasting_parameters)

StatementMeta(, , , Cancelled, )

In [ ]:
run = experiment.submit(automl_config)

StatementMeta(, , , Cancelled, )

In [ ]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Machine Learning portal: {}</a>".format(run.get_portal_url(), run.id))

StatementMeta(, , , Cancelled, )

In [ ]:
run.wait_for_completion()

import mlflow

# Get best model from automl run
best_run, non_onnx_model = run.get_output()

artifact_path = experiment_name + "_artifact"

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    # Save the model to the outputs directory for capture
    mlflow.sklearn.log_model(non_onnx_model, artifact_path)

    # Register the model to AML model registry
    mlflow.register_model("runs:/" + run.info.run_id + "/" + artifact_path, "kdsynapseworkspace-activitysummary-timeseries-Best")

StatementMeta(, , , Cancelled, )